# Problem 1

In [1]:
import numpy as np
import matplotlib.pyplot as pl
from scipy.spatial import KDTree

In [ ]:
rng = np.random.default_rng(66727)

In [ ]:
def gen_catalog(n, size=100, dims=3, rng=rng):
    '''
    Generate a catlog of 'n' points in 'dims' dimensions distributed uniformly at random

    n: number of samples
    size: side length of cubical box
    dims: dimensions of cube
    rng: numpy random number generator object
    '''
    return rng.random((n,dims))*size


def sq_dist(x1,x2):
    return np.sum((x1-x2)**2)

def sep(x1,x2):
    return (x1-x2) / np.sqrt(sq_dist(x1,x2))

def soft_grav(x1, x2, m = 1, eta = 1):
    return -m / (distf(x1,x2) + eta**2) * sep

def brute_acc(cat, f, eta=1):
    acc = np.zeros(cat.shape)

    for i in len(cat):
        for j in len(cat):
            if i<=j:
                continue
            a = f(cat[i], cat[j], eta=eta)
            acc[i] -= a
            acc[j] += a


def coms(node, cat, dic):
    if node isinstance(KDTree.leafnode):
        com = np.average(cat[node.idx], axis=0)
        dic[node] = com
        return(com)
    else:
        lcom = coms(node.less, cat)
        rcom = coms(node.greater, cat)
        dic[node] = lcom * node.less.children + rcom * node.greater.children

    return dic


def traverse(i, cat, node)



def tree_acc(cat, f):
    tree = KDTree(cat)
    dic = {}
    coms(tree.tree, cat, dic)
    acc = np.zeros(cat.shape)
    level = 0

    for i in range(len(cat)):
        unres = [tree.tree]
        for 



